In [2]:
import pandas as pd
import io
import ipywidgets as widgets
from IPython.display import display, HTML, FileLink, clear_output
from pathlib import Path
import pyexcel as pe
import zipfile

# Widget upload (tu dois cliquer sur "Choose Files" pour sélectionner)
upload = widgets.FileUpload(accept='.xls', multiple=True)
display(upload)

def clean_column_name(name):
    return "qte" if str(name).strip().lower() == "quantité" else name

converted_files = []

def is_real_xls(content_bytes):
    # Signature d'un vrai fichier XLS (Microsoft Compound File)
    return content_bytes[:4] == b'\xD0\xCF\x11\xE0'

def on_upload_change(change):
    clear_output(wait=True)
    display(upload)  # pour que le widget reste visible après upload

    if not upload.value:
        print("Aucun fichier sélectionné.")
        return

    converted_files.clear()

    for filedata in upload.value:
        filename = filedata['name']
        content_bytes = bytes(filedata['content'])  # conversion memoryview → bytes

        print(f"\nTraitement du fichier {filename}...")

        try:
            if is_real_xls(content_bytes):
                print("Détection : vrai fichier XLS.")
                book = pe.get_book(file_type="xls", file_stream=io.BytesIO(content_bytes))
                sheet = book[0]
                data = list(sheet.rows())
                if not data:
                    raise ValueError("Feuille vide")

                headers = data[0]
                rows = data[1:]
                df = pd.DataFrame(rows, columns=headers)

            else:
                print("Détection : fichier non XLS, lecture en CSV tabulé.")
                try:
                    text = content_bytes.decode('utf-8')
                except UnicodeDecodeError:
                    text = content_bytes.decode('cp1252')

                from io import StringIO
                df = pd.read_csv(StringIO(text), sep='\t')

            df.columns = [clean_column_name(col) for col in df.columns]

            for col in df.columns:
                df[col] = df[col].apply(lambda x: int(x) if isinstance(x, float) and x.is_integer() else x)

            output_filename = f"{Path(filename).stem}.csv"
            df.to_csv(output_filename, index=False, sep=';', encoding='cp1252', lineterminator='\r\n')
            print(f"Fichier CSV sauvegardé : {output_filename}, existe ? {Path(output_filename).exists()}")
            converted_files.append(output_filename)

            print(f"✅ Fichier converti : {output_filename}")

        except Exception as e:
            print(f"❌ Erreur pour {filename} : {e}")

    if converted_files:
        zip_name = "fichiers_convertis.zip"
        with zipfile.ZipFile(zip_name, 'w') as zipf:
            for file in converted_files:
                print(f"Ajout de {file} dans l'archive...")
                zipf.write(file, arcname=file)
        print(f"\n📦 Archive créée : {zip_name}")
        display(FileLink(zip_name))
        display(HTML(f'<a href="{zip_name}" target="_blank" download>Télécharger l\'archive ZIP</a>'))
    else:
        print("Aucun fichier converti, pas d'archive créée.")

upload.observe(on_upload_change, names='value')


FileUpload(value=(), accept='.xls', description='Upload', multiple=True)